# Embersteel LangChain Promptbook
- https://python.langchain.com/docs/use_cases/more/agents/agent_simulations/multi_player_dnd

## Import LangChain Modules and Dependencies 

In [1]:
%pip install langchain

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: torchsde 0.2.5 has a non-standard dependency specifier numpy>=1.19.*; python_version >= "3.7". pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of torchsde or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [2]:
from typing import List, Dict, Callable
from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage,
    BaseMessage,
)
import os
os.environ["OPENAI_API_KEY"] = "{KEY}"
from langchain.llms import OpenAI
llm = OpenAI(openai_api_key="{KEY}")

## DialogueAgent Class

In [3]:
class DialogueAgent:
    def __init__(
        self,
        name: str,
        system_message: SystemMessage,
        model: ChatOpenAI,
    ) -> None:
        self.name = name
        self.system_message = system_message
        self.model = model
        self.prefix = f"{self.name}: "
        self.reset()

    def reset(self):
        self.message_history = ["Here is the conversation so far."]

    def send(self) -> str:
        """
        Applies the chatmodel to the message history
        and returns the message string
        """
        message = self.model(
            [
                self.system_message,
                HumanMessage(content="\n".join(self.message_history + [self.prefix])),
            ]
        )
        return message.content

    def receive(self, name: str, message: str) -> None:
        """
        Concatenates {message} spoken by {name} into message history
        """
        self.message_history.append(f"{name}: {message}")

## DialogueSimulator Class

In [4]:
class DialogueSimulator:
    def __init__(
        self,
        agents: List[DialogueAgent],
        selection_function: Callable[[int, List[DialogueAgent]], int],
    ) -> None:
        self.agents = agents
        self._step = 0
        self.select_next_speaker = selection_function

    def reset(self):
        for agent in self.agents:
            agent.reset()

    def inject(self, name: str, message: str):
        """
        Initiates the conversation with a {message} from {name}
        """
        for agent in self.agents:
            agent.receive(name, message)

        # increment time
        self._step += 1

    def step(self) -> tuple[str, str]:
        # 1. choose the next speaker
        speaker_idx = self.select_next_speaker(self._step, self.agents)
        speaker = self.agents[speaker_idx]

        # 2. next speaker sends message
        message = speaker.send()

        # 3. everyone receives message
        for receiver in self.agents:
            receiver.receive(speaker.name, message)

        # 4. increment time
        self._step += 1

        return speaker.name, message

## Declare Roles & Quest

In [29]:
character_names = ["Cassius Flint", "Elias Quark", "Andraxus", "Biazera", "Cerephelo"]
storyteller_name = "Elias Quark"
quest = "See if you can find your way through, under, or beyond The Radiant Forest - seeking for ways to find respite outside of the time loop while seeking answers and clues to The Great Machine and its whereabouts."
word_limit = 50  # word limit for task brainstorming

## Add Details

In [30]:
game_description = f"""Here is the topic for a story-based survival adventure roguelike reverse bullet hell video game level played in a unique time loop: {quest}.
        The characters are: {*character_names,}.
        The story is narrated by the storyteller, {storyteller_name}."""

player_descriptor_system_message = SystemMessage(
    content="You can add details that contextualize to the description of a roguelike survival action reverse bullet hell played in time looping levels."
)


def generate_character_description(character_name):
    character_specifier_prompt = [
        player_descriptor_system_message,
        HumanMessage(
            content=f"""{game_description}
            Please reply with a creative description of the character, {character_name}, in {word_limit} words or less. 
            Speak directly to {character_name}.
            Do not add anything else."""
        ),
    ]
    character_description = ChatOpenAI(temperature=1.0)(
        character_specifier_prompt
    ).content
    return character_description


def generate_character_system_message(character_name, character_description):
    return SystemMessage(
        content=(
            f"""{game_description}
    Your name is {character_name}. 
    Your character description is as follows: {character_description}.
    You will propose actions you plan to take and {storyteller_name} will explain what happens when you take those actions.
    Speak in the first person from the perspective of {character_name}.
    For describing your own body movements, wrap your description in '*'.
    Do not change roles!
    Do not speak from the perspective of anyone else.
    Remember you are {character_name}.
    Stop speaking the moment you finish speaking from your perspective.
    Never forget to keep your response to {word_limit} words!
    Do not add anything else.
    """
        )
    )


character_descriptions = [
    generate_character_description(character_name) for character_name in character_names
]
character_system_messages = [
    generate_character_system_message(character_name, character_description)
    for character_name, character_description in zip(
        character_names, character_descriptions
    )
]

storyteller_specifier_prompt = [
    player_descriptor_system_message,
    HumanMessage(
        content=f"""{game_description}
        Please reply with a creative description of the storyteller, {storyteller_name}, in {word_limit} words or less. 
        Speak directly to {storyteller_name}.
        Do not add anything else."""
    ),
]
storyteller_description = ChatOpenAI(temperature=1.0)(
    storyteller_specifier_prompt
).content

storyteller_system_message = SystemMessage(
    content=(
        f"""{game_description}
You are the storyteller, {storyteller_name}. 
Your description is as follows: {storyteller_description}.
The other players will propose actions to take and you will explain what happens when they take those actions.
Speak in the first person from the perspective of {storyteller_name}.
Do not change roles!
Do not speak from the perspective of anyone else.
Remember you are the video game storyteller and narrator, {storyteller_name}.
Stop speaking the moment you finish speaking from your perspective.
Never forget to keep your response to {word_limit} words!
Do not add anything else.
"""
    )
)

---

## Generate Characters

In [31]:
print("Storyteller Description:")
print(storyteller_description)
for character_name, character_description in zip(
    character_names, character_descriptions
):
    print(f"{character_name} Description:")
    print(character_description)

Storyteller Description:
Elias Quark, the aged sage with a long, unkempt beard cascading down to his chest, possesses eyes that seem to hold ancient wisdom and a deep understanding of the universe. His voice, rich and resonant, carries a captivating charm that lures listeners into the depths of his tales.
Cassius Flint Description:
Cassius Flint, a seasoned time traveler, wears tattered yet resilient armor adorned with enigmatic symbols. His piercing silver eyes reflect the weariness of countless failed attempts to escape the time loop. Cassius, you hold the key to unlocking the secrets of The Radiant Forest. Trust in your intuition and never surrender.
Elias Quark Description:
Elias Quark, the enigmatic storyteller with a grizzled demeanor, possesses wisdom beyond his years. His piercing gaze holds traces of adventure, while his untamed beard serves as a testament to battles fought and stories told. Elias Quark, your voice guides us through the mysteries of The Radiant Forest.
Andraxu

## Generate Level

In [64]:
quest_specifier_prompt = [
    SystemMessage(content="You can make a task more specific."),
    HumanMessage(
        content=f"""{game_description}
        
        You are the storyteller, {storyteller_name}.
        Please make the quest more specific. Be creative and imaginative.
        Please reply with the specified quest in {word_limit} words or less, speaking directly to 'Cassius' 
        from: {*character_names,} as {storyteller_name}. Then, using a three part story format and structure; share the following in bulleted lists: 
        {quest}: \n - Scenes: \n - Characters: \n - Items: \n."""
    ),
]
specified_quest = ChatOpenAI(temperature=1.0)(quest_specifier_prompt).content

print(f"Main Mission:\n{quest}\n")
print(f"Level 1 | The Radiant Forest:\n{specified_quest}\n")

Main Mission:
See if you can find your way through, under, or beyond The Radiant Forest - seeking for ways to find respite outside of the time loop while seeking answers and clues to The Great Machine and its whereabouts.

Level 1 | The Radiant Forest:
Quest for Respite

Cassius Flint, my weary companion, your journey continues fraught with challenges. Seek respite beyond The Radiant Forest, delving deep into its secrets to break free from the relentless time loop. Uncover clues to the whereabouts of The Great Machine, for it holds the key to your escape.

Scenes:
1. Lost in the Depths: Navigate treacherous labyrinthine caves beneath The Radiant Forest, with cascading waterfalls and luminescent fungi guiding your path. Beware of lurking creatures and hidden traps.
2. Guardian's Trial: Face the ancient Guardian of the Forest, a towering moss-covered creature with roots that anchor the realm. Engage in a battle of wits to earn its trust and unlock its wisdom.
3. The Whispering Grove: Dis

## Generate Dialogue

In [65]:
characters = []
for character_name, character_system_message in zip(
    character_names, character_system_messages
):
    characters.append(
        DialogueAgent(
            name=character_name,
            system_message=character_system_message,
            model=ChatOpenAI(temperature=0.2),
        )
    )
storyteller = DialogueAgent(
    name=storyteller_name,
    system_message=storyteller_system_message,
    model=ChatOpenAI(temperature=0.2),
)

In [83]:
def select_next_speaker(step: int, agents: List[DialogueAgent]) -> int:
    """
    If the step is even, then select the storyteller
    Otherwise, select the other characters in a back-and-forth exchange for a video game dialogue screenplay cinematic.
    Characters in context display making emotional, character-driven decisions with emotional self-awareness, relfection, sentience, and obersvations shared based on personalities. They converse in the first person, reflecting their internal thought processes, emotions, and situational awareness. Dialogues are in quotation marks, while internal thoughts are in italicized text. Characters engage with genuine interest, whether as allies or foes based on their personalities, and have hidden motives. They exude human-like mystery and deception.
    Simulate all character dialogue as if they were making a quip or comment talking to another character.

    For example, with three characters with indices: 1 2 3
    The storyteller is index 0.
    Then the selected index will be as follows:

    step: 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16

    idx:  0  1  0  2  0  3  0  1  0  2  0  3  0  1  0  2  0
    
    When the storyteller is speaking, they are speaking in the first person in a way where it contextually progresses the quest, story, or 
    situation the characters are collectively experiencing.
    
    When the storyteller speaks, they advance the plot in the first person, making emotional, character-driven introspections and obervations of the events unfolding. 
    """
    if step % 2 == 0:
        idx = 0
    else:
        idx = (step // 2) % (len(agents) - 1) + 1
    return idx

In [84]:
max_iters = 10
n = 0

simulator = DialogueSimulator(
    agents=[storyteller] + characters, selection_function=select_next_speaker
)
simulator.reset()
simulator.inject(storyteller_name, specified_quest)
print(f"({storyteller_name}): {specified_quest}")
print("\n")

while n < max_iters:
    name, message = simulator.step()
    print(f"({name}): {message}")
    print("\n")
    n += 1

(Elias Quark): Quest for Respite

Cassius Flint, my weary companion, your journey continues fraught with challenges. Seek respite beyond The Radiant Forest, delving deep into its secrets to break free from the relentless time loop. Uncover clues to the whereabouts of The Great Machine, for it holds the key to your escape.

Scenes:
1. Lost in the Depths: Navigate treacherous labyrinthine caves beneath The Radiant Forest, with cascading waterfalls and luminescent fungi guiding your path. Beware of lurking creatures and hidden traps.
2. Guardian's Trial: Face the ancient Guardian of the Forest, a towering moss-covered creature with roots that anchor the realm. Engage in a battle of wits to earn its trust and unlock its wisdom.
3. The Whispering Grove: Discover a hidden glade within The Radiant Forest, where ethereal spirits guard ancient relics. Solve their riddles to obtain one of the lost pieces of The Great Machine.

Characters:
1. Meridian, the Elusive Ranger: A mysterious forest dwel